In [2]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')


In [5]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
classes = 'Yes_Drone No_Drone'.split()
for i in classes:
    pathlib.Path(f'img_data/{i}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'{i}'):
        songname = f'{i}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB')
        plt.axis('off')
        plt.savefig(f'img_data/{i}/{filename[:-3].replace(".", "")}.png')
        plt.clf()        

<Figure size 720x720 with 0 Axes>

In [54]:
header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
header += ' label'
header = header.split()

In [55]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
classes = 'Yes_Drone No_Drone'.split()
for i in classes:
    for filename in os.listdir(f'{i}'):
        songname = f'{i}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        to_append += f' {i}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())


In [56]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,label
0,B_S2_D1_067-bebop_000_.wav,0.605145,2390.183296,2130.183683,5018.671875,0.125966,Yes_Drone
1,B_S2_D1_067-bebop_001_.wav,0.526925,3183.232455,2058.200449,5713.955078,0.228668,Yes_Drone
2,B_S2_D1_067-bebop_002_.wav,0.607370,3024.987931,2156.907712,5685.244141,0.197255,Yes_Drone
3,B_S2_D1_067-bebop_003_.wav,0.631430,3055.299767,2163.135084,5715.869141,0.194195,Yes_Drone
4,B_S2_D1_067-bebop_004_.wav,0.597745,3267.331920,2100.423525,5808.719576,0.236804,Yes_Drone


In [57]:
data.shape

(2735, 7)

In [58]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)


In [59]:
label_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(label_list)


In [60]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [62]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(2, activation='softmax'))

In [63]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [64]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
2188/2188 [==============================] - 0s 129us/step - loss: 0.5103 - accuracy: 0.7655
Epoch 2/20
2188/2188 [==============================] - 0s 28us/step - loss: 0.4276 - accuracy: 0.8158
Epoch 3/20
2188/2188 [==============================] - 0s 24us/step - loss: 0.4063 - accuracy: 0.8236
Epoch 4/20
2188/2188 [==============================] - 0s 26us/step - loss: 0.3979 - accuracy: 0.8272
Epoch 5/20
2188/2188 [==============================] - 0s 25us/step - loss: 0.4022 - accuracy: 0.8176
Epoch 6/20
2188/2188 [==============================] - 0s 27us/step - loss: 0.3958 - accuracy: 0.8245
Epoch 7/20
2188/2188 [==============================] - 0s 28us/step - loss: 0.3814 - accuracy: 0.8300
Epoch 8/20
2188/2188 [==============================] - 0s 27us/step - loss: 0.3753 - accuracy: 0.8304
Epoch 9/20
2188/2188 [==============================] - 0s 29us/step - loss: 0.3698 - accuracy: 0.8355
Epoch 10/20
2188/2188 [==============================] - 0s 28us/step - 

In [65]:
test_loss, test_acc = model.evaluate(X_test,y_test)

547/547 [==============================] - 0s 225us/step


In [66]:
print('test_loss: ',test_loss)
print('test_acc: ',test_acc)

test_loss:  0.33140150458982703
test_acc:  0.8720292448997498


In [67]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [68]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 1988 samples, validate on 200 samples
Epoch 1/30
1988/1988 [==============================] - 0s 240us/step - loss: 0.6410 - accuracy: 0.6363 - val_loss: 0.5678 - val_accuracy: 0.7700
Epoch 2/30
1988/1988 [==============================] - 0s 43us/step - loss: 0.5110 - accuracy: 0.7832 - val_loss: 0.5343 - val_accuracy: 0.7950
Epoch 3/30
1988/1988 [==============================] - 0s 41us/step - loss: 0.4438 - accuracy: 0.8144 - val_loss: 0.5656 - val_accuracy: 0.7650
Epoch 4/30
1988/1988 [==============================] - 0s 40us/step - loss: 0.4310 - accuracy: 0.8164 - val_loss: 0.5563 - val_accuracy: 0.7450
Epoch 5/30
1988/1988 [==============================] - 0s 41us/step - loss: 0.4132 - accuracy: 0.8244 - val_loss: 0.5217 - val_accuracy: 0.7600
Epoch 6/30
1988/1988 [==============================] - 0s 41us/step - loss: 0.4021 - accuracy: 0.8229 - val_loss: 0.4919 - val_accuracy: 0.7750
Epoch 7/30
1988/1988 [==============================] - 0s 42us/step - loss: 0.395

In [69]:
results

[0.34751122443506005, 0.866544783115387]

In [70]:
import pickle

In [71]:
with open('scaler.pickle', 'wb') as f:
    pickle.dump(scaler, f)

In [72]:
model.save('CNN.h5')